**Проект: VK_AI-помощник для образования (Auto ET)**

**Разработка RAG-системы**

Работу выполнил: Высоцкий Денис

In [ ]:
#! pip install langchain
#! pip install langchain_community
#! pip install gigachat
#! pip install sentence-transformers
#! pip install faiss-cpu
#!pip install pymupdf

In [ ]:
import os
import torch
import numpy as np
from getpass import getpass
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics.pairwise import cosine_similarity
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import GigaChat
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import VectorStoreRetriever
import fitz
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer, util
import json
from datasets import Dataset
import tqdm
import requests

import warnings
warnings.filterwarnings('ignore')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

Для работы c гигачатом API был использован secret key

[Сбер для разработчиков](https://developers.sber.ru/portal/products/gigachat-api#tariffs)

In [ ]:
os.environ["GIGACHAT_CREDENTIALS"] = getpass()

··········


Обертка для удобства общения с моделью:

In [ ]:
def giga_answer(template: str) -> str:
    '''
    Функция для генерации ответа по промпту
    '''
    llm = GigaChat(verify_ssl_certs=False,
                   #max_tokens=64
    )

    # создаем промпт
    prompt = PromptTemplate.from_template(template)
    llm_chain = prompt | llm

    return llm_chain.invoke(input={})

In [ ]:
giga_answer('Что такое покрытие товаров?')

'Покрытие товаров – это термин, используемый для описания уровня защиты, предоставляемой производителем товара покупателям и потребителям от возможных рисков, связанных с качеством, безопасностью или дефектами продукции. \n\n### Основные виды покрытия товаров:\n1. **Гарантийное'

Зальем документы

In [118]:
url = "https://www.dropbox.com/scl/fi/fsny9pb5iirbst9wozy70/ML_notes.pdf?rlkey=mlanfbrgg0ceuz34ikt7y95h5&st=ohzffwa8&dl=1"
output_path = "ML_notes.pdf"

response = requests.get(url)
with open(output_path, "wb") as f:
    f.write(response.content)

In [121]:
doc = fitz.open("ML_notes.pdf")

corpus_docs = [
    Document(
        page_content=page.get_text(),
        metadata={"page_number": i}
    )
    for i, page in enumerate(doc)
]
print("Документов:", len(corpus_docs))
print(corpus_docs[0].page_content[:200], "…")

Документов: 186
Лекция 1
Введение в машинное обучение
Е. А. Соколов
ФКН ВШЭ
5 сентября 2021 г.
В последнее время стало появляться всё больше данных. Университеты зна-
ют всё про своих студентов — оценки и достижения  …


Разделим на чанки

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, chunk_overlap=100
)

docs = splitter.split_documents(corpus_docs)
print("Всего чанков:", len(docs))

Чанков: 850


В качестве эмбеддера использовался `ruElectra-small`. Векторная БД - `FAISS`

In [ ]:
embed_model = HuggingFaceEmbeddings(model_name='ai-forever/ruElectra-small')

In [ ]:
db = FAISS.from_documents(docs, embed_model) # строим векторную базу

db.index.ntotal # документов в нашей базе

850

Экспериментальный промпт

In [ ]:
query = '''
Что такое покрытие товаров?
'''

docs = db.similarity_search(query)
docs #

[Document(id='8c9ac398-e6e9-487f-b610-9842edfb87fd', metadata={'page_number': 61}, page_content='но:\nhamming(a, X) = 1\nℓ\nℓ\nX\ni=1\n|Yi \\ Zi| + |Zi \\ Yi|\nK\n.'),
 Document(id='6865acbd-b986-4085-b174-313bac2bc6a3', metadata={'page_number': 125}, page_content='ших степеней. Например, отношение веса при мономе степени m−1 к весу при мономе\nпервой степени равно\ns\nCm−1\nm\nR\nC1mRm−1 =\nr\n1\nRm−2,'),
 Document(id='8ed610e1-a2a9-4a42-afd0-1f9346152408', metadata={'page_number': 126}, page_content='ца\nG =\n\x14\nexp\n\x12\n−∥xi −xj∥2\n2σ2\n\x13\x15ℓ\ni,j=1\nявляется невырожденной при σ > 0.'),
 Document(id='61d07351-2f37-4d91-b9fd-35008f820744', metadata={'page_number': 182}, page_content='только самые популярные товары, а большая часть ассортимента игнорируется. В\nкачестве простейшей метрики можно использовать покрытие каталога, которые вы-\nчисляется как доля товаров, порекомендованных хотя бы один раз.')]

Эксперимент с системным промптом

In [ ]:
retriever = VectorStoreRetriever(vectorstore=db)

system_prompt = (
    '''
    Ты помощник для студента.
    Тебе задают вопрос, а ты должен правильно на него ответить.
    Также тебе будет даны документы, из которых
    ты должен брать нужную информацию и с ее помощью отвечать на вопросы.
    Если в предоставленных текстах нет ответа, отвечай по точному шаблону:
    Невозможно однозначно ответить на данный вопрос

    Информация из текстов: {context}
    '''
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
llm = GigaChat(verify_ssl_certs=False,
               #max_tokens=64 #не успел разобраться, как грамотно
               #выставить ограничения для органического ответа
               )
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

query = '''
Что такое покрытие товаров?
'''

chain.invoke({"input": query})

{'input': '\nЧто такое покрытие товаров?\n',
 'context': [Document(id='8c9ac398-e6e9-487f-b610-9842edfb87fd', metadata={'page_number': 61}, page_content='но:\nhamming(a, X) = 1\nℓ\nℓ\nX\ni=1\n|Yi \\ Zi| + |Zi \\ Yi|\nK\n.'),
  Document(id='6865acbd-b986-4085-b174-313bac2bc6a3', metadata={'page_number': 125}, page_content='ших степеней. Например, отношение веса при мономе степени m−1 к весу при мономе\nпервой степени равно\ns\nCm−1\nm\nR\nC1mRm−1 =\nr\n1\nRm−2,'),
  Document(id='8ed610e1-a2a9-4a42-afd0-1f9346152408', metadata={'page_number': 126}, page_content='ца\nG =\n\x14\nexp\n\x12\n−∥xi −xj∥2\n2σ2\n\x13\x15ℓ\ni,j=1\nявляется невырожденной при σ > 0.'),
  Document(id='61d07351-2f37-4d91-b9fd-35008f820744', metadata={'page_number': 182}, page_content='только самые популярные товары, а большая часть ассортимента игнорируется. В\nкачестве простейшей метрики можно использовать покрытие каталога, которые вы-\nчисляется как доля товаров, порекомендованных хотя бы один раз.')],
 'answer': 

Помимо ответа, подтянулись еще и фрагменты источников, которые использовались при генерации. Для удобства обернем через функцию:

In [ ]:
def giga_rag_answer(query):
    return chain.invoke({"input": query})['answer']

In [ ]:
giga_rag_answer('Что такое покрытие товаров?')

'Покрытие товаров — это метрика, которая показывает долю товаров из общего ассортимента, которые были рекомендованы хотя бы один раз. Проще говоря, это процент товаров, попавших в список рекомендаций.'

In [ ]:
giga_rag_answer('Что такое рекомендательная система?')

'Рекомендательная система – это алгоритм, который анализирует предпочтения пользователя (например, какие товары он покупал ранее или что смотрел) и предлагает ему новые элементы, которые могут быть интересны на основе его предыдущей активности. Эти системы широко используются в интернет-магазинах, стриминговых сервисах, социальных сетях и других платформах для повышения удобства пользователей и увеличения продаж.'

In [ ]:
giga_rag_answer('Что такое новизна в рекомендательных системамх?')

'В контексте рекомендательных систем новизна (или freshness) относится к тому, насколько недавно добавленный или обновленный контент предлагается пользователям. Рекомендательные системы стремятся показывать пользователям наиболее актуальные и свежие элементы, что помогает поддерживать интерес пользователей и улучшать их опыт взаимодействия с системой.'

Валидация RAG c попощью метрики accuracy, основанной на косинусном расстоянии идеального ответа и модельного

In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#сет из 20 проверенных вопросов
testset = [
    {"question": "Что такое покрытие товаров?", "answers": ["доля товаров, порекомендованных хотя бы один раз"]},
    {"question": "Что такое алгоритм один против всех?", "answers": ["метод классификации, при котором каждый класс рассматривается как отдельный бинарный классификатор"]},
    {"question": "Какие есть методы оптимизации второго порядка?", "answers": ["метод Ньютона, квази-Ньютоновские методы"]},
    {"question": "На чем основан DBSCAN?", "answers": ["алгоритм основан на плотности кластеров, где кластеры формируются на основе плотности точек в пространстве"]},
    {"question": "Как работает стекинг?", "answers": ["метод ансамблирования, в котором прогнозы алгоритмов объявляются новыми признаками, и поверх них обучается ещё один алгоритм (который называют мета-алгоритмом)"]},
    {"question": "Что такое новизна в контексте рекомендательных систем?", "answers": ["доля новых для пользователя товаров среди рекомендованных"]},
    {"question": "Что такое задача кластеризации?", "answers": ["задача разделения объектов на группы, обладающие некоторыми свойствами. Примером может служить кластеризация документов из электронной библиотеки"]},
    {"question": "Какие есть способы оценки градиента?", "answers": ["Есть несколько способов: Стохастический градиентный спуск SGD, средний стохастический градиен SAG, а также модификации: Метод инерции (momentum), AdaGrad и RMSprop, Adam"]},
    {"question": "Какое основное свойство логистической регрессии?", "answers": ["Основным свойством является тот факт, что метод корректно оценивает вероятность принадлежности объекта к каждому из классов."]},
    {"question": "Для чего используется метод главных компонент?", "answers": ["Часто возникает задача уменьшения размерности признакового пространства."]},
    {"question": "Для чего используется метод Isolation Forest?", "answers": ["Подход для обнаружения аномалий."]},
    {"question": "Что такое факторизационная машина?", "answers": ["Модель, позволяющая строить рекомендательные модели на основе большого количества категориальных и вещественных признаков."]},
    {"question": "Какую задачу решает EM-алгоритм?", "answers": ["Решает задачу максимизации полного правоподобия путем попеременной оптимизации по параметрам и по скрытым переменным."]},
    {"question": "Какие модели обучения представлений (representation learning) рассматриваются в документах?", "answers": ["Разберается word2vec - способ обучения представлений для слов."]},
    {"question": "Как называется модель, в которой зависимость ищется как линейная комбинация нелинейных функций от выборки?", "answers": ["Такая модель называется называется линейная модель над базисными функциями"]},
    {"question": "Какие есть популярные методы построения деревьев?", "answers": ["ID3 (использует энтропийный критерий), C4.5 (использует критерий Gain Ratio - нормированный энтропийный критерий), CART (использует критерий Джини)."]},
    {"question": "Что такое F-мера?", "answers": ["Метрика качества, равная гармоническому среднему между precision и recall"]},
    {"question": "Что такое регуляризация?", "answers": ["Штраф для контроля сложности модели"]},
    {"question": "Недостатки использования аналитической формулы для весов линейной регрессии?", "answers": ["Обращение матрицы - сложная операция; матрица может быть вырожденной"]},
    {"question": "Что такое градиент функции?", "answers": ["Это вектор его частных производных"]}
    # не проверял наличие следующей информации в документе:
    #{"question": "Как работает градиентный бустинг?", "answers": ["метод ансамблирования, где новые модели обучаются для коррекции ошибок предыдущих моделей"]},
    #{"question": "Что такое переобучение модели?", "answers": ["ситуация, когда модель слишком хорошо подстраивается под обучающий набор данных и теряет способность обобщать на новых данных"]},
    #{"question": "Что такое рекуррентные нейронные сети?", "answers": ["тип нейронных сетей, которые используют циклические соединения для обработки последовательных данных"]},
    #{"question": "Как работает алгоритм случайного леса?", "answers": ["метод ансамблирования, при котором несколько деревьев решений обучаются на случайных подмножествах данных, а их результаты комбинируются"]},
    #{"question": "Что такое метод главных компонент?", "answers": ["метод понижения размерности, который ищет новые переменные, называемые главными компонентами, которые объясняют наибольшую часть вариации в данных"]},
    #{"question": "Что такое градиентный спуск?", "answers": ["алгоритм оптимизации, который используется для минимизации функции потерь, итерационно обновляя параметры модели по направлению отрицательного градиента"]},
    #{"question": "Что такое регуляризация в машинном обучении?", "answers": ["метод, используемый для предотвращения переобучения, который добавляет штраф за сложность модели"]},
    #{"question": "Какие бывают типы машинного обучения?", "answers": ["обучение с учителем, обучение без учителя, обучение с подкреплением"]},
    #{"question": "Что такое метод наименьших квадратов?", "answers": ["метод оптимизации, используемый для нахождения параметров, минимизирующих сумму квадратов отклонений между предсказаниями и истинными значениями"]},
    #{"question": "Как работает метод опорных векторов?", "answers": ["метод классификации, который ищет оптимальную гиперплоскость, разделяющую данные на классы с максимальным зазором"]},
    #{"question": "Что такое ансамблирование моделей?", "answers": ["метод, при котором несколько моделей комбинируются для улучшения общей производительности и повышения точности"]},
    #{"question": "Что такое кластеризация?", "answers": ["метод машинного обучения, цель которого — разделить данные на группы (кластеры), где элементы внутри группы более похожи друг на друга"]},
    #{"question": "Как работает алгоритм K-means?", "answers": ["метод кластеризации, который разделяет данные на K кластеров, минимизируя внутрикластерное расстояние"]},
    #{"question": "Что такое методы линейной регрессии?", "answers": ["методы, которые моделируют зависимость между переменными с помощью линейной функции"]},
    #{"question": "Что такое нейронная сеть?", "answers": ["модель машинного обучения, состоящая из нейронов, организованных в слои, и используемая для решения различных задач, таких как классификация и регрессия"]},
    #{"question": "Что такое обучение без учителя?", "answers": ["метод обучения, при котором модель пытается найти структуру в данных без использования меток"]},
    #{"question": "Что такое сверточная нейронная сеть?", "answers": ["тип нейронной сети, который используется для обработки изображений и других данных с сеточной структурой, применяя свертки"]},
    #{"question": "Что такое метод ближайшего соседа?", "answers": ["метод классификации, в котором объект классифицируется на основе классов ближайших к нему объектов"]},
    #{"question": "Что такое методы оптимизации?", "answers": ["методы, используемые для поиска наилучших параметров модели, которые минимизируют ошибку на обучающем наборе"]},
    #{"question": "Что такое вычислительная сложность алгоритма?", "answers": ["характеристика, описывающая количество ресурсов (время или память), необходимых для выполнения алгоритма"]},
    #{"question": "Что такое рекуррентная нейронная сеть?", "answers": ["тип нейронной сети, в которой используются циклические соединения для работы с последовательными данными"]},
    #{"question": "Что такое сверточная нейронная сеть?", "answers": ["нейронная сеть, которая использует свертки для обработки изображений и видеопотоков"]},
    #{"question": "Что такое обобщение в машинном обучении?", "answers": ["способность модели давать точные прогнозы на данных, которые она не видела во время обучения"]},
    #{"question": "Что такое байесовский классификатор?", "answers": ["метод классификации, основанный на теореме Байеса, который оценивает вероятности классов на основе данных"]},
    #{"question": "Как работает метод случайного леса?", "answers": ["метод ансамблирования, при котором несколько деревьев решений обучаются на случайных подмножествах данных, а их результаты комбинируются"]},
    #{"question": "Что такое проблема многократного коллизии?", "answers": ["проблема, возникающая в алгоритмах поиска, когда два элемента данных оказываются в одной и той же ячейке в хеш-таблице"]},
    #{"question": "Как работает метод главных компонент?", "answers": ["метод уменьшения размерности, который находит новые переменные (главные компоненты), которые объясняют наибольшую часть вариации в данных"]},
    #{"question": "Что такое перцептрон?", "answers": ["простая модель нейронной сети, используемая для решения задач классификации"]},
    #{"question": "Что такое поддержка гиперплоскости?", "answers": ["пространство, которое разделяет данные между двумя классами в методах опорных векторов"]},
    #{"question": "Как работает многослойный перцептрон?", "answers": ["нейронная сеть, состоящая из нескольких слоев, которая может решать сложные задачи классификации"]},
    #{"question": "Как работает алгоритм градиентного спуска?", "answers": ["алгоритм оптимизации, который минимизирует функцию потерь путем корректировки параметров модели на каждом шаге"]},
    #{"question": "Что такое регрессия?", "answers": ["метод машинного обучения, используемый для предсказания числовых значений"]},
    #{"question": "Что такое машинное обучение?", "answers": ["метод, при котором алгоритмы обучаются на данных для выполнения задач без явного программирования"]},
    #{"question": "Что такое нейронная сеть?", "answers": ["модель машинного обучения, которая состоит из слоев нейронов, имитирующих работу человеческого мозга"]},
    #{"question": "Что такое градиентный бустинг?", "answers": ["метод ансамблирования, при котором слабые модели комбинируются в одну сильную модель"]},
    #{"question": "Что такое обучение с подкреплением?", "answers": ["метод машинного обучения, при котором агент обучается на основе вознаграждений и наказаний за свои действия"]},
    #{"question": "Что такое обучение с учителем?", "answers": ["метод машинного обучения, при котором модель обучается на размеченных данных"]},
    #{"question": "Что такое метод наименьших квадратов?", "answers": ["метод оптимизации, который минимизирует сумму квадратов отклонений между предсказанными и фактическими значениями"]},
    #{"question": "Как работает линейная регрессия?", "answers": ["метод статистического анализа, который находит линейную зависимость между переменными"]},
    #{"question": "Что такое случайный лес?", "answers": ["метод ансамблирования, который использует несколько деревьев решений для повышения точности классификации"]}
]


In [ ]:
correct, total = 0, 0
threshold = 0.5  # повышение порога несправедливо ухудшало accuracy;
                 # считай 0.5 - эвристика

print("Тестирование:\n")
for sample in tqdm.tqdm(testset):
    q, gold = sample["question"], sample["answers"]

    pred = giga_rag_answer(q)
    gold_text = gold[0]  # можно расширить под несколько синонимичных вариантов

    # косинусное расстояние между эмбеддингами
    emb_pred = model.encode(pred, convert_to_tensor=True)
    emb_gold = model.encode(gold_text, convert_to_tensor=True)

    similarity = float(util.pytorch_cos_sim(emb_pred, emb_gold))

    print(f"Вопрос: {q}")
    print(f"Ожидаемый ответ: {gold_text}")
    print(f"Ответ модели: {pred}")
    print(f"Сходство: {similarity:.2f}")

    if similarity >= threshold:
        correct += 1
    total += 1

print(f"Semantic Accuracy@{total}: {correct/total:.2%}")


Тестирование:



  5%|▌         | 1/20 [00:02<00:44,  2.33s/it]

Вопрос: Что такое покрытие товаров?
Ожидаемый ответ: доля товаров, порекомендованных хотя бы один раз
Ответ модели: Покрытие товаров — это метрика, которая показывает долю товаров, рекомендованных хотя бы один раз, от общего количества товаров в каталоге. Эта метрика помогает оценить, насколько широко представлены товары в системе рекомендаций. Чем выше значение покрытия товаров, тем больше различных товаров было предложено пользователям.
Сходство: 0.57


 10%|█         | 2/20 [00:02<00:22,  1.25s/it]

Вопрос: Что такое алгоритм один против всех?
Ожидаемый ответ: метод классификации, при котором каждый класс рассматривается как отдельный бинарный классификатор
Ответ модели: Из предоставленной информации невозможно однозначно ответить на данный вопрос.
Сходство: 0.10


...

Вопрос: Что такое регуляризация?
Ожидаемый ответ: Штраф для контроля сложности модели
Ответ модели: Регуляризация – это метод, используемый в машинном обучении и статистике для уменьшения переобучения модели. Он добавляет дополнительное слагаемое к функции ошибки, чтобы минимизировать сложность модели, тем самым улучшая ее обобщающую способность. В результате регуляризация помогает избежать чрезмерной подгонки модели к обучающим данным и повышает точность предсказаний на новых данных.
Сходство: 0.50


 95%|█████████▌| 19/20 [00:19<00:00,  1.34it/s]

Вопрос: Недостатки использования аналитической формулы для весов линейной регрессии?
Ожидаемый ответ: Обращение матрицы - сложная операция; матрица может быть вырожденной
Ответ модели: Невозможно однозначно ответить на данный вопрос, так как в предоставленных текстах нет информации о недостатках использования аналитической формулы для весов линейной регрессии.
Сходство: 0.24


100%|██████████| 20/20 [00:20<00:00,  1.04s/it]

Вопрос: Что такое градиент функции?
Ожидаемый ответ: Это вектор его частных производных
Ответ модели: Градиент функции является вектором, который указывает направление наибольшего роста функции в данной точке. В многомерном случае градиент определяется как вектор частных производных функции по каждой переменной. Для функции \( f(x_1, x_2, ..., x_n) \) градиент можно записать как:

\[
\nabla f = \left( \frac{\partial f}{\partial x_1}, \frac{\partial f}{\partial x_2}, ..., \frac{\partial f}{\partial x_n} \right)
\]

Градиент используется в различных областях математики, таких как оптимизация, где он помогает находить направления наискорейшего возрастания функции.
Сходство: 0.62
Semantic Accuracy@20: 45.00%


Получили качество: 45% accuracy.

Основными причинами такого результата являются: эмбеддинги, не улавливающие смысл, и неоптимальные размеры чанков.

Дальнейшее развитие и улучшение качества RAG:


*   бОльшее использование мета-данных документа (титулы, подзаголовки)
*   использование более продвинутого эмбеддера и БД, и в последнюю очередь LLM

